# Shipwrecked

In this project, we will write a Python program that uses Bayes' Rule to quickly find a solitary fisherman who has gone missing. As the director of the Coast Guard's search and rescue operations for the region, we've already interviewed the fisherman's wife and <b>determined his last known position</b>, now more than six hours old.
He radioed that he was abandoning ship, but no one knows if he's in a life raft or floating in the sea. The waters around the cape are warm, but if he's immersed, he'll experience hypothermia in approximately 12 hours. If he's wearing a floatation device and is lucky, he might last three days.

### The Objective
We will create a search and rescue game that uses Bayes' Rule to inform player choices on how to conduct a search.

### The Strategy
We'll start with initial target probabilities for the sailor's location and update them for the search results. If we achieve an effective search of an area but find nothing, the probability that the sailor is in another area will increase.
But, just as in real life, there are two ways things could go wrong: you thoroughly search an area but still miss the sailor, or your search goes poorly, wasting a day's effort. To equate this to search effectiveness scores, in the first case, you might get an SEP of 0.85, but the sailor is in the remaining 15% of the area not searched. In the second case, your SEP is 0.2, and you've left 80 percent of the area unsearched.

To aid the player, we will use the OpenCV library to build an interface for working with the program. Although the interface can be something simple like a menu built in the shell, you'll also want a map of the cape and the search areas. The OpenCV library is an excellent choice for this game, since it lets you display images and add drawings and text.

In [ ]:
#importing the necessary packages
import sys, random, itertools, numpy as np, cv2 as cv

MAP_FILE = 'cape_python.png'

SA1_CORNERS = (130, 265, 180, 315) #(UL-X, UL-Y, LR-X, LR-Y)
SA2_CORNERS = (80, 255, 130, 305)
SA3_CORNERS = (105, 205, 155, 255)

#Defining the Search Class
class Search():
    """Bayesian Search & Rescue game """

    def __init__(self, name):
        self.name = name
        self.img = cv.imread(MAP_FILE, cv.IMREAD_COLOR)
        if self.img is None:
            print("Could not load map file {}".format(MAP_FILE), file=sys.stderr)
            sys.exit(1)
        
        self.area_actual = 0
        self.sailor_actual = [0, 0] #As "local" coords within search area

        self.sa1 = self.img[SA1_CORNERS[1] : SA1_CORNERS[3], SA1_CORNERS[0] : SA1_CORNERS[2]]
        self.sa2 = self.img[SA2_CORNERS[1] : SA2_CORNERS[3], SA2_CORNERS[0] : SA2_CORNERS[2]]
        self.sa3 = self.img[SA3_CORNERS[1] : SA3_CORNERS[3], SA3_CORNERS[0] : SA3_CORNERS[2]]

        self.p1 = 0.2
        self.p2 = 0.5
        self.p3 = 0.3

        self.sep1 = 0
        self.sep2 = 0
        self.sep3 = 0
    
    def draw_map(self, last_known):
        """Display basemap with scale, last known xy location, search areas"""
        cv.line(self.img, (20, 370), )